## Convert input data from S-JTSK to WGS-84

Let's download address points from State Administration of Land Surveying and Cadastre of the Czech republic http://nahlizenidokn.cuzk.cz/stahniadresnimistaruian.aspx

Data are stored in the form of CVS file in [data/20180430_OB_573108_ADR.csv](data/20180430_OB_573108_ADR.csv)


In [6]:
with open("data/20180430_OB_573108_ADR.csv") as indata:
    for line in indata.readlines()[0:10]:
        line = line.strip()
        print(line)

Kód ADM;Kód obce;Název obce;Kód MOMC;Název MOMC;Kód MOP;Název MOP;Kód části obce;Název části obce;Kód ulice;Název ulice;Typ SO;Číslo domovní;Číslo orientační;Znak čísla orientačního;PSČ;Souřadnice Y;Souřadnice X;Platí Od
17444373;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;1;;;50723;684386.77;1019544.23;2013-12-01T00:00:00
17444381;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;2;;;50723;684361.18;1019551.52;2013-12-01T00:00:00
17444390;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;3;;;50723;684337.58;1019554.31;2013-12-01T00:00:00
17444403;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;4;;;50723;684313.31;1019553.91;2013-12-01T00:00:00
17444411;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;5;;;50723;684304.98;1019560.71;2013-12-01T00:00:00
17444420;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;6;;;50723;684257.89;1019492.65;2013-12-01T00:00:00
17444438;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;7;;;50723;684261.67;1019582.20;2013-12-01T00:00:00
17444446;573108;Libáň;;;;;81663;Kozodírky;;;č.p.;8;;;50723;684248.97;1019311.12

Parse the input lines

In [74]:
data = {}
with open("data/20180430_OB_573108_ADR.csv") as indata:
    for line in indata.readlines()[1:]:
        line = line.strip()
 
        (code_adm, city, name, code_momc, name_momc, code_mop,
         name_mop, code_part, name_part, code_street, name_street,
         type_so, house_number, house_orientation, char, zipcode, Y, X, valid) = line.split(";")
        
        if not Y and not X:
            continue
        
        data[code_adm] = {
            "street": name_street,
            "number": house_number,
            "city": name,
            "zip": zipcode,
            "Y": -1*float(Y),
            "X": -1*float(X)
        }

NOTE: `Y` and `X` coordinates in S-JTSK are `x` and `y` in cartesian system.

NOTE: S-JTSK uses positive values, but EPSG:5514 assumes negative values, therefore `x = -1*Y`

Let's convert `x` and `y` coordinates from czech local [S-JTSK](http://epsg.io/5514) to global [WGS-84](http://epsg.io/4326) system.

### Transformation using PyProj

[PyProj](https://pypi.org/project/pyproj/) is Python bindings to [Proj.4](https://proj4.org/) library.


In [75]:
from pyproj import Proj, transform
import pyproj

original = Proj("+proj=krovak +lat_0=49.5 +lon_0=24.83333333333333 +alpha=30.28813972222222 +k=0.9999 +x_0=0 +y_0=0 +ellps=bessel +pm=greenwich +units=m +no_defs +towgs84=570.8,85.7,462.8,4.998,1.587,5.261,3.56")

destination = Proj(init='EPSG:4326')

for adm in data:
    x, y = (data[adm]["Y"], data[adm]["X"])
    trans_coords = transform(original, destination, x, y)
    print(trans_coords)
    data[adm]["x"] = trans_coords[0]
    data[adm]["y"] = trans_coords[1]


(15.189795303177085, 50.365577521669046)
(15.190165026949463, 50.365541526408315)
(15.190499026034642, 50.365543406193545)
(15.19083671176722, 50.36557449447179)
(15.190964911124967, 50.36552329486614)
(15.191500837814553, 50.36618367628235)
(15.191606873529683, 50.36538074630013)
(15.191303411474077, 50.36781274910129)
(15.190815842294825, 50.36812073996246)
(15.190711453943562, 50.368276645892735)
(15.190672990642222, 50.368435919231864)
(15.190612239040584, 50.36860253322098)
(15.190218098803673, 50.36872345622789)
(15.190576706135902, 50.36925169557342)
(15.190599458463447, 50.369652570754)
(15.190506072532164, 50.37003892635392)
(15.190541183018, 50.37031420236054)
(15.190382710925673, 50.3706843900245)
(15.190156733865246, 50.371278642890736)
(15.19031465253598, 50.37087134438805)
(15.191766013498361, 50.36496357813856)
(15.191048996793493, 50.36996508180242)
(15.191228501414429, 50.364941345021904)
(15.191580406038103, 50.36459649433218)
(15.191578390833198, 50.3648119289706)
(1

### Creating new GeoJSON file

Let's create new [GeoJSON](http://geojson.org) vector structure with address points

In [76]:
import json

features = {
    "type": "FeatureCollection",
    "features":[]
}

for adm in data:
    feature = {
        "type":"Feature",
        "properties": {
            "city": data[adm]["city"],
            "zip": data[adm]["zip"],
            "street": data[adm]["street"],
            "number": data[adm]["number"]
        },
        "geometry": {
            "type": "Point",
            "coordinates": [data[adm]["x"], data[adm]["y"]]
        }
    }
    
    features["features"].append(feature)

print(features["features"][0:5])

[{'type': 'Feature', 'properties': {'city': 'Libáň', 'zip': '50723', 'street': '', 'number': '1'}, 'geometry': {'type': 'Point', 'coordinates': [15.189795303177085, 50.365577521669046]}}, {'type': 'Feature', 'properties': {'city': 'Libáň', 'zip': '50723', 'street': '', 'number': '2'}, 'geometry': {'type': 'Point', 'coordinates': [15.190165026949463, 50.365541526408315]}}, {'type': 'Feature', 'properties': {'city': 'Libáň', 'zip': '50723', 'street': '', 'number': '3'}, 'geometry': {'type': 'Point', 'coordinates': [15.190499026034642, 50.365543406193545]}}, {'type': 'Feature', 'properties': {'city': 'Libáň', 'zip': '50723', 'street': '', 'number': '4'}, 'geometry': {'type': 'Point', 'coordinates': [15.19083671176722, 50.36557449447179]}}, {'type': 'Feature', 'properties': {'city': 'Libáň', 'zip': '50723', 'street': '', 'number': '5'}, 'geometry': {'type': 'Point', 'coordinates': [15.190964911124967, 50.36552329486614]}}]


Write it to output GeoJSON file

In [77]:
import json

with open("../../outputs/addresses.geojson", "w") as out:
    json.dump(features, out)

And download the data for QGIS: [../../outputs/addresses.geojson](../../outputs/addresses.geojson)

![](images/addresses.png)

---
[<- Introduction](01_projections.ipynb) | [GeoDjango ->](../06_geodjango/00_introduction.ipynb)